In [1]:
import selenium
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import warnings
warnings.filterwarnings('ignore')
import time
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

### 1. Scrape the details of most viewed videos on YouTube from Wikipedia.
Url = https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos You need to find following details:
* A) Rank
* B) Name
* C) Artist
* D) Upload date
* E) Views

In [19]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos')
soup = BeautifulSoup(driver.page_source, 'lxml')
tables = soup.find_all('table')
most_viewed = pd.read_html(driver.page_source)[1].set_index('No.').iloc[:,:4]
most_viewed.head(5)

,Video name,Uploader,Views (billions),Publication date
No.,,,,
1.,"""Baby Shark Dance""[4]",Pinkfong Baby Shark - Kids' Songs & Stories,11.98,"June 17, 2016"
2.,"""Despacito""[7]",Luis Fonsi,8.04,"January 12, 2017"
3.,"""Johny Johny Yes Papa""[14]",LooLoo Kids,6.56,"October 8, 2016"
4.,"""Bath Song""[15]",Cocomelon – Nursery Rhymes,5.88,"May 2, 2018"
5.,"""Shape of You""[16]",Ed Sheeran,5.87,"January 30, 2017"


### 
2. Scrape the details team India’s international fixtures from bcci.tv. Url = https://www.bcci.tv/.
You need to find following details:
* A) Match title (I.e. 1st ODI)
* B) Series
* C) Place
* D) Date
* E) Time
* Note: - From bcci.tv home page you have reach to the international fixture page through code.

In [58]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.bcci.tv/international/fixtures')

In [59]:
while True:
    try:
        loadMoreButton = driver.find_element_by_xpath("/html/body/div[2]/div[2]/div/div/div/div[2]/div[3]/div[2]/div/button")
        time.sleep(2)
        loadMoreButton.click()
        time.sleep(5)
    except Exception as e:
        print (e)
        break
print ("Complete")
time.sleep(10)

Message: element click intercepted: Element <button class="match-btn btn-red d-flex align-items-center justify-content-center mx-auto mt-3" ng-click="viewMoreMatches()">...</button> is not clickable at point (508, 695). Other element would receive the click: <div class="cookie no-disply" style="display: block;">...</div>
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	Ordinal0 [0x0084ACD3+2075859]
	Ordinal0 [0x007DEE61+1633889]
	Ordinal0 [0x006DB7BD+571325]
	Ordinal0 [0x00711499+791705]
	Ordinal0 [0x0070F4AC+783532]
	Ordinal0 [0x0070D0AB+774315]
	Ordinal0 [0x0070BD37+769335]
	Ordinal0 [0x00701C76+728182]
	Ordinal0 [0x0072731C+881436]
	Ordinal0 [0x007015BF+726463]
	Ordinal0 [0x00727534+881972]
	Ordinal0 [0x0073B56A+963946]
	Ordinal0 [0x00727136+880950]
	Ordinal0 [0x006FFEFD+720637]
	Ordinal0 [0x00700F3F+724799]
	GetHandleVerifier [0x00AFEED2+2769538]
	GetHandleVerifier [0x00AF0D95+2711877]
	GetHandleVerifier [0x008DA03A+521194]
	GetHandleVerifier [0x008D8DA0+516432]
	Ord

In [55]:
desc = driver.find_elements(By.XPATH, '//div[contains(@class,"fixture-card ng-scope border-orange") or contains(@class,"fixture-card ng-scope border-sky") or contains(@class,"fixture-card ng-scope border-green")]')
data = pd.DataFrame([i.text for i in desc], columns = ['desc'])
data = data['desc'].str.split('\n', expand=True).rename(columns = {0:'Series', 1:'Date', 2:'Time', 3:'Team_1', 5:'Team_2', 6:'place'})
data.drop([4,7], axis = 1, inplace = True)


In [62]:
place= data['place'].str.split('-', expand=True).rename(columns = {0:'Match_Title', 1:'Place'}).drop([2], axis = 1)
BCCI_data = pd.concat([data, place], axis = 1).drop(['place'], axis = 1)

,Series,Date,Time,Team_1,Team_2,Match_Title,Place
0,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,29 DEC 2022,5:15 PM IST,India Women U19,South Africa Women U19,2nd T20I,"Steyn City School Ground, Pretoria"
1,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,31 DEC 2022,1:30 PM IST,India Women U19,South Africa Women U19,3rd T20I,"Steyn City School Ground, Pretoria"
2,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,2 JAN 2023,5:15 PM IST,India Women U19,South Africa Women U19,4th T20I,"Steyn City School Ground, Pretoria"
3,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,3 JAN 2023,7:00 PM IST,India,Sri Lanka,1st T20I,"Wankhede Stadium, Mumbai"
4,INDIA WOMEN U19 TOUR OF SOUTH AFRICA WOMEN U19,4 JAN 2023,1:30 PM IST,India Women U19,South Africa Women U19,5th T20I,"Steyn City School Ground, Pretoria"
5,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,5 JAN 2023,7:00 PM IST,India,Sri Lanka,2nd T20I,"Maharashtra Cricket Association Stadium, Pune"
6,SRI LANKA TOUR OF INDIA T20 SERIES 2022-23,7 JAN 2023,7:00 PM IST,India,Sri Lanka,3rd T20I,"Saurashtra Cricket Association Stadium, Rajkot"
7,SRI LANKA TOUR OF INDIA ODI SERIES 2022-23,10 JAN 2023,1:30 PM IST,India,Sri Lanka,1st ODI,"Barsapara Cricket Stadium, Guwahati"
8,SRI LANKA TOUR OF INDIA ODI SERIES 2022-23,12 JAN 2023,1:30 PM IST,India,Sri Lanka,2nd ODI,"Eden Gardens, Kolkata"
9,SRI LANKA TOUR OF INDIA ODI SERIES 2022-23,15 JAN 2023,1:30 PM IST,India,Sri Lanka,3rd ODI,"Greenfield International Stadium, Thiruvanant..."


### 
3. Scrape the details of selenium exception from guru99.com. Url = https://www.guru99.com/
You need to find following details:
* A) Name
* B) Description
* Note: - From guru99 home page you have to reach to selenium exception handling page through code.

In [32]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.guru99.com/')
time.sleep(10)
search = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/main/div/article/div/div[1]/div[1]/div[2]/div/div[1]/div/div/div/form/table/tbody/tr/td[1]/div/table/tbody/tr/td[1]/input')
search.send_keys('selenium exception handling')
butn = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/main/div/article/div/div[1]/div[1]/div[2]/div/div[1]/div/div/div/form/table/tbody/tr/td[2]/button')
butn.click()
time.sleep(5)
click = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div/main/div/article/div/div[1]/div[1]/div[2]/div/div[1]/div/div/div/div[1]/div[6]/div[2]/div/div/div[1]/div[1]/div[1]/div[1]/div/a')
click.click()

In [42]:
[i.text.split(':') for i in driver.find_elements(By.XPATH,"//div[@class='entry-content single-content']/p")][:40]

[]

In [50]:
pd.DataFrame([i.text for i in driver.find_elements(By.XPATH, '//div[@class="entry-content single-content"]//p')])[:40]

,0
0,We make tons of efforts to take boredom out of...
1,
2,
3,
4,
5,
6,
7,
8,
9,


In [45]:
pd.DataFrame([i.text for i in driver.find_elements(By.TAG_NAME, 'p')])

,0
0,We make tons of efforts to take boredom out of...
1,
2,
3,
4,
5,
6,
7,
8,
9,


### 
4 Scrape the details of State-wise GDP of India from statisticstime.com. Url = http://statisticstimes.com/
You have to find following details:
* A) Rank
* B) State
* C) GSDP(18-19)- at current prices
* D) GSDP(19-20)- at current prices
* E) Share(18-19)
* F) GDP($ billion)

In [73]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.statisticstimes.com/economy/india/indian-states-gdp.php')
soup = BeautifulSoup(driver.page_source, 'lxml')
#tables = soup.find_all('table')
df = pd.read_html(driver.page_source)[1]

In [76]:
gdp_india = df.iloc[:, 0:6]


In [79]:
gdp_india.head(5)

Rank          State GSDP (Cr INR at Current prices)            Share  \
  Rank          State                           19-20    18-19   18-19   
0  1.0    Maharashtra                               -  2632792  13.94%   
1  2.0     Tamil Nadu                         1845853  1630208   8.63%   
2  3.0  Uttar Pradesh                         1687818  1584764   8.39%   
3  4.0        Gujarat                               -  1502899   7.96%   
4  5.0      Karnataka                         1631977  1493127   7.91%   

  GDP ($billion)  
            2019  
0        399.921  
1        247.629  
2        240.726  
3        228.290  
4        226.806

In [72]:
df.loc[:2, (slice(None),'19-20')]

,GSDP (Cr INR at Current prices),GSDP (Cr INR at 2011-12 prices)
,19-20,19-20
0,-,-
1,1845853,1312929
2,1687818,1166817


### 
5 Scrape the details of trending repositories on Github.com. Url = https://github.com/
You have to find the following details:
* A) Repository title
* B) Repository description
* C) Contributors count
* D) Language used

* Note: - From the home page you have to click on the trending option from Explore menu through code.

In [164]:

driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://github.com/')
time.sleep(5)
butn = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/header/div/div[2]/div/nav/ul/li[3]/button')
butn.click()
butn = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/header/div/div[2]/div/nav/ul/li[3]/div/ul[3]/li[3]/a')
butn.click()

In [165]:
repo_url = []
Count = []


for page in range(1):
    url = driver.find_elements(By.XPATH, '//h1[@class="h3 lh-condensed"]//a')
    
    for i in url:
        repo_url.append(i.get_attribute('href'))

In [166]:
len(repo_url)

25

In [167]:
Lang = []
Title = []
Desc = []

In [168]:
for url in repo_url:
    driver.get(url)
    time.sleep(2)
   
    try:
        count = driver.find_element(By.XPATH, '/html/body/div[1]/div[4]/div/main/turbo-frame/div/div/div/div[3]/div[2]/div/div[5]/div/h2/a')
        Count.append(count.text)
        lang = driver.find_element(By.XPATH,'/html/body/div[1]/div[4]/div/main/turbo-frame/div/div/div/div[3]/div[2]/div/div[6]/div/ul')
        Lang.append(lang.text)
        title = driver.find_element(By.XPATH,'/html/body/div[1]/div[4]/div/main/div/div[1]/div/div/span[1]/a')
        Title.append(title.text)
        desc = driver.find_element(By.XPATH,'/html/body/div[1]/div[4]/div/main/turbo-frame/div/div/div/div[3]/div[1]/readme-toc/div/div[2]/article/p[1]')
        Desc.append(desc.text)
        
    except NoSuchElementException:
        Count.append('_')
        Lang.append('_')
        Title.append('_')
        Desc.append('_')

In [174]:
Count = pd.DataFrame(Count, columns = ['Contribution']).iloc[1:,:]
Lang = pd.DataFrame(Lang, columns = ['Lang'])
Title = pd.DataFrame(Title, columns = ['Title'])
Desc = pd.DataFrame(Desc, columns = ['Desccription'])

In [178]:
github_data = pd.concat([Count, Lang, Title, Desc], axis = 1)

In [179]:
github_data

,Contribution,Lang,Title,Desccription
0,NaN,_,_,_
1,_,_,_,_
2,Contributors 262,Shell\n100.0%,papers-we-love,
3,_,_,_,_
4,Contributors 2,Jupyter Notebook\n79.4%\nPython\n20.6%,karpathy,"The simplest, fastest repository for training/..."
5,_,_,_,_
6,_,_,_,_
7,Contributors 10,TypeScript\n64.3%\nRust\n26.9%\nShell\n5.0%\nP...,zlib-searcher,
8,Contributors 11,Python\n100.0%,TencentARC,
9,Contributors 100,_,_,_


### 
6 Scrape the details of top 100 songs on billiboard.com. Url = https:/www.billboard.com/
You have to find the following details:
* A) Song name
* B) Artist name
* C) Last week rank
* D) Peak rank
* E) Weeks on board
* Note: - From the home page you have to click on the charts option then hot 100-page link through code.

In [103]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.billboard.com/')
#time.sleep(5)
butn = driver.find_element(By.XPATH, '/html/body/div[3]/header/div/div[2]/div/div/div[2]/div[2]/div/div/nav/ul/li[1]/a')
butn.click()
time.sleep(5)
button = driver.find_element(By.XPATH, '/html/body/div[3]/main/div[2]/div[3]/div/div/div[2]/div/div[1]/a')
button.click()


In [116]:
desc = driver.find_elements(By.XPATH,'//li[@class="lrv-u-width-100p"]')
rank = driver.find_elements(By.XPATH,'//span[@class="c-label  a-font-primary-bold-l u-font-size-32@tablet u-letter-spacing-0080@tablet"]')

In [132]:
rank = pd.DataFrame([i.text for i in rank], columns = ['Rank'])

In [131]:
data = pd.DataFrame([i.text for i in desc], columns = ['Col'])
Song = data = data['Col'].str.split('\n', expand = True).rename({0:'Song_Name', 1:'Artist_Name', 2:'Last_Weak_Rank', 3:'Peak_Weak_Rank', 4:'Week_Onboard'}, axis = 1)

In [136]:
Song_Data = pd.concat([Song,rank], axis = 1)
Song_Data

,Song_Name,Artist_Name,Last_Weak_Rank,Peak_Weak_Rank,Week_Onboard,Rank
0,All I Want For Christmas Is You,Mariah Carey,1,1,58,1
1,Rockin' Around The Christmas Tree,Brenda Lee,2,2,52,2
2,Jingle Bell Rock,Bobby Helms,3,3,49,3
3,Last Christmas,Wham!,5,4,31,4
4,A Holly Jolly Christmas,Burl Ives,4,4,32,5
...,...,...,...,...,...,...
95,Pick Me Up,Gabby Barrett,-,69,19,96
96,SOS,SZA,79,32,3,97
97,Forget Me,Lewis Capaldi,-,95,7,98
98,La Jumpa,Arcangel & Bad Bunny,-,68,2,99


### 
7. Scrape the details of Data science recruiters from naukri.com. Url = https://www.naukri.com/
You have to find the following details:
* A) Name
* B) Designation
* C) Company
* D) Skills they hire for
* E) Location
* Note: - From naukri.com homepage click on the recruiters option and the on the search pane type Data science and click on search. All this should be done through code

In [41]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.naukri.com/hr-recruiters-consultants')
rec = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[2]/div/div/div/ul/li[1]/a')
rec.click()
time.sleep(5)
search = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[1]/form/div[1]/div/div[1]/div[1]/div[2]/input')

search.send_keys('Data Scientist')
buttn = driver.find_element(By.XPATH, '/html/body/div[2]/div[2]/div[1]/div[1]/form/div[1]/button')
buttn.click()

In [45]:
name_c = driver.find_elements(By.XPATH, '//p[@class="highlightable"]')
data = pd.DataFrame([i.text for i in name_c], columns = ['Name'])
data = data['Name'].str.split('\n', expand=True).rename(columns = {0:'Name', 1:'Designation', 2:'Company', 3:'Location'})

In [70]:
skills = driver.find_elements(By.XPATH, '//div[@class="hireSec highlightable"]')
skills = pd.DataFrame([i.text for i in skills], columns = ['Skill'])
Recruiter_Data = pd.concat([data, skills], axis = 1)

In [75]:
Recruiter_Data.head(5)

,Name,Designation,Company,Location,Skill
0,Tanushree,Lead Recruiter,RecRoots,Noida,"UI Developers, Software Engineers, Quality Ass..."
1,sashi bhushan,Senior Specialist,Wipro,Bengaluru / Bangalore,"pig, hbase, sas, spss, apache, python, nosql, ..."
2,Bitapi,HR and Operation Manager,Anvaya,Bengaluru / Bangalore,"Hadoop, Big Data, Data Scientists, Java, Sprin..."
3,Gajendra Singh,Head of Recruitment - Product &amp;...,A Leading of Product Start-UP Company,Noida,"Data Scientist, Big Data, Hadoop, Web Analytic..."
4,Sadashiv Kulkarni,Company HR,Digitially Insights Pvt Ltd,Pune,"Big Data Engineer, Data Scientist, Solutions, ..."


### 
8. Scrape the details of Highest selling novels.
* Url = https://www.theguardian.com/news/datablog/2012/aug/09/best-selling-books-all-time-fifty-shades-greycompare/
You have to find the following details:
* A) Book name
* B) Author name
* C) Volumes sold
* D) Publisher
* E) Genre

In [2]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.theguardian.com/news/datablog/2012/aug/09/best-selling-books-all-time-fifty-shades-grey-compare')

In [12]:
soup = BeautifulSoup(driver.page_source, 'lxml')
tables = soup.find_all('table')
book_data = pd.read_html(driver.page_source)[0].iloc[:, 1:]

,Title,Author,Volume Sales,Publisher,Genre
0,"Da Vinci Code,The","Brown, Dan",5094805,Transworld,"Crime, Thriller & Adventure"
1,Harry Potter and the Deathly Hallows,"Rowling, J.K.",4475152,Bloomsbury,Children's Fiction
2,Harry Potter and the Philosopher's Stone,"Rowling, J.K.",4200654,Bloomsbury,Children's Fiction
3,Harry Potter and the Order of the Phoenix,"Rowling, J.K.",4179479,Bloomsbury,Children's Fiction
4,Fifty Shades of Grey,"James, E. L.",3758936,Random House,Romance & Sagas
...,...,...,...,...,...
96,Happy Days with the Naked Chef,"Oliver, Jamie",794201,Penguin,Food & Drink: General
97,"Hunger Games,The:Hunger Games Trilogy","Collins, Suzanne",792187,Scholastic Ltd.,Young Adult Fiction
98,"Lost Boy,The:A Foster Child's Search for the L...","Pelzer, Dave",791507,Orion,Biography: General
99,Jamie's Ministry of Food:Anyone Can Learn to C...,"Oliver, Jamie",791095,Penguin,Food & Drink: General


### 
9. Scrape the details most watched tv series of all time from imdb.com.
* Url = https://www.imdb.com/list/ls095964455/
* You have to find the following details:
* A) Name
* B) Year span
* C) Genre
* D) Run time
* E) Ratings
* F) Votes

In [77]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://www.imdb.com/list/ls095964455/')

In [82]:
name = driver.find_elements(By.XPATH, '//div[@class="lister-item-content"]')
data = pd.DataFrame([i.text for i in name], columns = ['Desc'])
data = data['Desc'].str.split('\n', expand=True)

In [98]:
run_g = data[1].str.split('|', expand=True).rename(columns = {0:'unknown', 1:'run', 2:'genre'}).iloc[:,1:]
name_y = data[0].str.split('.', expand=True).rename(columns = {1:'name'}).iloc[:,1:2]
name_y = name_y['name'].str.split('(', expand=True).rename(columns = {0:'name', 1:'year'})
votes = data[6].str.split(':', expand=True).rename(columns = {0:'name', 1:'votes'}).iloc[:,1:]
rating = data[[2]].rename(columns = {2:'rating'})

In [128]:
Top_100 = pd.concat([run_g, name_y, votes, rating], axis = 1)
Top_100

,run,genre,name,year,votes,rating
0,57 min,"Action, Adventure, Drama",Game of Thrones,2011–2019),"2,100,527",9.2
1,51 min,"Drama, Fantasy, Horror",Stranger Things,2016– ),"1,190,808",8.7
2,44 min,"Drama, Horror, Thriller",The Walking Dead,2010–2022),"994,358",8.1
3,60 min,"Drama, Mystery, Thriller",13 Reasons Why,2017–2020),"294,278",7.5
4,43 min,"Drama, Mystery, Sci-Fi",The 100,2014–2020),"253,379",7.6
...,...,...,...,...,...,...
95,42 min,Drama,Reign,2013–2017),"50,382",7.4
96,50 min,"Adventure, Comedy, Drama",A Series of Unfortunate Events,2017–2019),"62,001",7.8
97,42 min,"Crime, Drama, Mystery",Criminal Minds,2005– ),"199,994",8.1
98,45 min,"Comedy, Crime, Drama",Scream: The TV Series,2015–2019),"41,717",7.1


### 
10 Details of Datasets from UCI machine learning repositories.
* Url = https://archive.ics.uci.edu/
* You have to find the following details:
* A) Dataset name
* B) Data type
* C) Task
* D) Attribute type
* E) No of instances
* F) No of attribute
* G) Year
* Note: - from the home page you have to go to the ShowAllDataset page through code.

In [2]:
driver = webdriver.Chrome(r'chromedriver.exe')
driver.get('https://archive.ics.uci.edu/ml/index.php')
search = driver.find_element(By.XPATH, '/html/body/table[1]/tbody/tr/td[2]/span[2]/a')
search.click()

In [8]:
soup = BeautifulSoup(driver.page_source, 'lxml')
tables = soup.find_all('td') 
data = pd.read_html(driver.page_source)[5]

In [98]:
data = data.rename({0:'Name'}, axis = 1)
data = data.dropna(subset=['Name']).rename({1:'Data Types', 2:'Default Task', 3:'Attribute Types', 4:' Instances', 5:'Attributes',6:'Year'}, axis = 1).iloc[1:,:]

In [99]:
data

,Name,Data Types,Default Task,Attribute Types,Instances,Attributes,Year
1,Abalone,Multivariate,Classification,"Categorical, Integer, Real",4177,8,1995
3,Adult,Multivariate,Classification,"Categorical, Integer",48842,14,1996
5,Annealing,Multivariate,Classification,"Categorical, Integer, Real",798,38,NaN
7,Anonymous Microsoft Web Data,NaN,Recommender-Systems,Categorical,37711,294,1998
9,Arrhythmia,Multivariate,Classification,"Categorical, Integer, Real",452,279,1998
...,...,...,...,...,...,...,...
1235,Influenza outbreak event prediction via Twitte...,Multivariate,Classification,"Integer, Real",75840,525,2020
1237,Turkish Music Emotion Dataset,Multivariate,Classification,"Integer, Real",400,50,2020
1239,Maternal Health Risk Data Set,NaN,Classification,NaN,1014,7,2020
1241,Room Occupancy Estimation,"Multivariate, Time-Series",Classification,Real,10129,16,2021
